In [34]:
import matplotlib as mpl
mpl.use('pgf')

mpl.rcParams.update({
    'pgf.texsystem': 'pdflatex',
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

import matplotlib.pyplot as plt
# Show the image with ipywidges
import matplotlib.dates as mdates

import pandas as pd
import datetime
import json
import numpy as np

In [35]:
dark = 0
red = 1
amber = 2
green = 3
redamber = 4

def state_to_color(state: int):
    if state == 0:
        return (0, 0, 0) # dark
    elif state == 1:
        return (195, 0, 0) # red
    elif state == 2:
        return (255, 193, 7) # amber
    elif state == 3:
        return (40, 167, 69) # green
    elif state == 4:
        return (255, 132, 0) # redamber
    elif state == 5:
        return (255, 255, 0)
    elif state == 6:
        return (43, 255, 0)
    elif state == 9:
        return (221, 0, 255)
    elif state == 10:
        return (0, 0, 255)
    else:
        raise ValueError("Invalid state")
    
def cycles_to_colors(cycles: list):
    return list(map(state_to_color, cycles))

In [3]:
# def cycle_dependent_instability(cycles: list) -> int:
#     if len(cycles) < 2:
#         raise ValueError("Not enough cycles")
#     diffs = []
#     for i in range(len(cycles) - 1):
#         cycle_a, cycle_b = cycles[i], cycles[i + 1]
#         diff = 0
#         for j in range(max(len(cycle_a), len(cycle_b))):
#             if j >= len(cycle_a) or j >= len(cycle_b):
#                 diff += 1
#             elif  cycle_a[j] != cycle_b[j]:
#                 diff += 1
#         diffs.append(diff)
#     return int(np.median(diffs))

# def cycle_independent_instability(cycles: list) -> float:
#     if len(cycles) < 2:
#         raise ValueError("Not enough cycles")
#     cycles_flat = [item for sublist in cycles for item in sublist]
#     greens_end_times = []
#     greens_start_times = []
#     for i in range(len(cycles_flat) - 1):
#         if cycles_flat[i] == green and cycles_flat[i + 1] != green:
#             greens_end_times.append(i)
#         if cycles_flat[i] != green and cycles_flat[i + 1] == green:
#             greens_start_times.append(i)
#     wait_times = []
#     for i in range(min(len(greens_end_times), len(greens_start_times))):
#         wait_times.append(greens_start_times[i] - greens_end_times[i])
#     if len(wait_times) == 0:
#         return 0
#     return len(set(wait_times)) / len(wait_times)

In [4]:
# # Load a random example for bottom_left, bottom_right, top_left, top_right
# import os
# import random

# def load_example(prefix, index):
#     if "right" in prefix:
#         dir = "data/adaptiveness/case_studies_random_right_25/"
#     else:
#         dir = "data/adaptiveness/case_studies_random/"
#     files = os.listdir(dir)
#     files = [f for f in files if f.startswith(prefix)]
#     files = [f for f in files if f.endswith(".json")]
#     example = files[index]
#     with open(dir + example) as f:
#         data = json.load(f)
#     return data

# def to_state_arr(input_str):
#     """Example: [3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]"""
#     input_str = input_str.replace("[", "")
#     input_str = input_str.replace("]", "")
#     input_str = input_str.replace(" ", ",")
#     return list(map(int, input_str.split(",")))


In [46]:
# def plot_cycles(cycles, hlines, ax):
#     longest_cycle = max(map(len, cycles))
#     cycles = [cycle + [0] * (longest_cycle - len(cycle)) for cycle in cycles]
#     aspect = (longest_cycle / len(cycles)) * 0.8
#     ax.imshow([cycles_to_colors(cycle) for cycle in cycles], interpolation='none', rasterized=True, aspect=aspect)
#     for hline in hlines:
#         ax.axhline(hline, color='blue', linewidth=0.5)
#     ax.set_xticks([])
#     ax.set_yticks([])

# for section in ["bottom_left", "bottom_right", "top_left", "top_right"]:
#     fig, axs = plt.subplots(2, 2, figsize=(5, 4))
#     for i, ax in enumerate(axs.flatten()):
#         example = load_example(section, i)
#         example_cycles = [to_state_arr(cycle['Results']) for cycles in example["Cycles"] for cycle in cycles]

#         hlines = []
#         cycle_idx = 0
#         for cycle_arr in example["Cycles"]:
#             hlines.append(cycle_idx)
#             cycle_idx += len(cycle_arr)
            
#         plot_cycles(example_cycles, hlines, ax)

#     plt.subplots_adjust(wspace=0.05, hspace=0.05)
#     fig_name = f"figures/predictability-rw-example-{section}"
#     plt.savefig(f'{fig_name}.pdf', bbox_inches='tight')
#     plt.savefig(f'{fig_name}.png', bbox_inches='tight', dpi=300)

In [47]:
import os
import random
import json

with open('data/predictability/processed_things_2024_01_22.json') as f:
    processed_things = json.load(f)
    
erroneous_things = []

for thing_name, thing in processed_things.items():
    clean_name = thing_name.replace("_primary", "")
    clean_name = clean_name.replace("_secondary", "")
    if thing["TotalCyclesCount"] == 0:
        continue
    if thing["TotalRemovedCycleCount"] / thing["TotalCyclesCount"] > 0.1:
        erroneous_things.append(clean_name)
        
print(len(erroneous_things))

dir = "data/predictability/case_studies_random/"
files = os.listdir(dir)

def to_state_arr(input_str):
    """Example: [3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]"""
    input_str = input_str.replace("[", "")
    input_str = input_str.replace("]", "")
    input_str = input_str.replace(" ", ",")
    return list(map(int, input_str.split(",")))

def load_example(i):
    random.seed(i)
    example = random.choice(files)
    with open(dir + example) as f:
        data = json.load(f)
    return data

def plot_cycles(data, ax):
    cycles = [to_state_arr(cycle['Results']) for cycles in data["Cycles"] for cycle in cycles]
    longest_cycle = max(map(len, cycles))
    cycles = [cycle + [10] * (longest_cycle - len(cycle)) for cycle in cycles]
    aspect = (longest_cycle / len(cycles)) * 0.6
    ax.imshow([cycles_to_colors(cycle) for cycle in cycles], interpolation='none', rasterized=True, aspect=aspect)
    ax.set_xticks([])
    ax.set_yticks([])
    cycle_discrepancy = data["Metrics"]
    wait_time_diversity = data["ShiftsFuzzyness"]
    ax.set_title(f"({(cycle_discrepancy):.0f}s, {(wait_time_diversity * 100):.0f}%)", fontsize=10)

n_rows = 15
n_cols = 6

fig, axs = plt.subplots(figsize=(5, 11), ncols=n_cols, nrows=n_rows)

cells = []
seed = 0
for i, ax in enumerate(axs.flatten()):
    valid_example_found = False
    while not valid_example_found:
        example = load_example(seed)
        if example["Name"] not in erroneous_things and example not in cells and example["Metrics"] != -1 and example["ShiftsFuzzyness"] != -1:
            valid_example_found = True
        seed += 1
    cells.append(example)

for i, ax in enumerate(axs.flatten()):
    example = cells[i]
    plot_cycles(example, ax)

    plt.subplots_adjust(left=0, right=1, top=1, bottom=0, wspace=0.05, hspace=0.1)

    fig_name = f"figures/predictability-case-studies"

for i in range(n_rows):
    axs[i][0].set_ylabel(f'{i+1}')

axs[n_rows - 1][0].set_xlabel('A')
axs[n_rows - 1][1].set_xlabel('B')
axs[n_rows - 1][2].set_xlabel('C')
axs[n_rows - 1][3].set_xlabel('D')
axs[n_rows - 1][4].set_xlabel('E')
axs[n_rows - 1][5].set_xlabel('F')

plt.savefig(f'{fig_name}.pdf', bbox_inches='tight')
plt.savefig(f'{fig_name}.png', bbox_inches='tight', dpi=300)
plt.savefig(f'{fig_name}.pgf', bbox_inches='tight', dpi=300)

2106
